# Content Moderator

This demo use **Microsoft Azure Cognitive Services**.

In [1]:
import time 
import urllib
import requests
import cv2
import operator
import numpy as np
from __future__ import print_function
import matplotlib.pyplot as plt
%matplotlib inline 

_url = 'https://westus.api.cognitive.microsoft.com/contentmoderator/moderate/v1.0/ProcessImage/Evaluate'
_key = '__KEY__'
_maxNumRetries = 10

In [2]:
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result

In [3]:
from IPython.display import Image as ipImage, display

urlImage = 'https://i0.wp.com/starwarsanon.files.wordpress.com/2015/06/leia-rotj-1.png'

img = ipImage(url=urlImage, width=200)
display(img)

In [4]:
headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 
    "DataRepresentation":"URL",
    "Value": urlImage
} 
data = None
params = None

result = processRequest( json, data, headers, params )

In [5]:
import json

print(json.dumps(result, indent=2, sort_keys=True))

{
  "AdultClassificationScore": 2.699400702965704e-09,
  "AdvancedInfo": [
    {
      "Key": "ImageDownloadTimeInMs",
      "Value": "27"
    },
    {
      "Key": "ImageSizeInBytes",
      "Value": "214105"
    }
  ],
  "IsImageAdultClassified": false,
  "IsImageRacyClassified": true,
  "RacyClassificationScore": 0.996896505355835,
  "Result": true,
  "Status": {
    "Code": 3000,
    "Description": "OK",
    "Exception": null
  },
  "TrackingId": "WE_ibiza_07ff23b7-65c7-4ecb-be86-edb4544069e8_ContentModerator.F0_62344901-6faf-48cb-b11e-b4342a93cbb9"
}
